# 1. Importing modules and functions

In [3]:
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
import rdkit
print(rdkit.__version__)
%pylab inline
from molvs import standardize_smiles
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score
import joblib
import pickle
from numpy import savetxt
from IPython.display import HTML
from rdkit.Chem import PandasTools

2024.09.1
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
def convert_smi_to_canon_smi(smi):
    
    try:
        canon_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi),isomericSmiles = False)
    except:
        canon_smi='wrong_smiles'
    return canon_smi

In [5]:
def standart(smi):
    global m
    if smi!='wrong_smiles':
        try:
            smiles=standardize_smiles(smi)
            m = Chem.MolFromSmiles(smi)
        except:
            smi='error kekule'
    else:
        m = 'check the smiles'
    return m

# MORGAN FP_Gradient Boosting Rat intravenous LD50, mg/kg  

## Load data and curation work set

In [8]:
# Set file path
df_ws=pd.read_csv('datasets/rat_LD50_WS.csv')
df_ws

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical_SMILES,InChIKey,rat_intravenous_LD50
0,TOX-3990,178,acetamide,CC(N)=O,CC(N)=O,DLFVBJFMPXGRIB-UHFFFAOYSA-N,0.674442
1,TOX-2407,180,propan-2-one,CC(C)=O,CC(C)=O,CSCPPACGZOOCGX-UHFFFAOYSA-N,1.023664
2,TOX-8193,12025,pyrrolidin-2-one,O=C1CCCN1,O=C1CCCN1,HNJBEVLQSNELDL-UHFFFAOYSA-N,1.072628
3,TOX-4318,1030,"propane-1,2-diol",CC(O)CO,CC(O)CO,DNIAPMSPPWPWGF-UHFFFAOYSA-N,1.073618
4,TOX-8477,99937,"2,3,3a,5,6,6a-hexahydrofuro[3,2-b]furan-3,6-diol",OC1COC2C(O)COC12,OC1COC2C(O)COC12,KLDXJTOLSGUMSJ-UHFFFAOYSA-N,1.123382
...,...,...,...,...,...,...,...
1853,TOX-43158,116224,2-diethoxyphosphorylsulfanylethyl-ethyl-methyl...,CCOP(=O)(OCC)SCC[S+](C)CC,CCOP(=O)(OCC)SCC[S+](C)CC,YDYNMVNUGKEYOS-UHFFFAOYSA-N,7.232647
1854,TOX-2075,14955,"9,10,21,25-tetramethoxy-15,15,30,30-tetramethy...",COc1ccc2cc1Oc1cc3c(cc1OC)CC[N+](C)(C)C3Cc1ccc(...,COc1ccc2cc1Oc1cc3c(cc1OC)CC[N+](C)(C)C3Cc1ccc(...,JFXBEKISTKFVAB-UHFFFAOYSA-N,7.270733
1855,TOX-43154,116216,2-diethoxyphosphorylsulfanylethyl(diethyl)sulf...,CCOP(=O)(OCC)SCC[S+](CC)CC,CCOP(=O)(OCC)SCC[S+](CC)CC,QACHTXIQQRWDTM-UHFFFAOYSA-N,7.458497
1856,TOX-43157,116222,2-dimethoxyphosphorylsulfanylethyl-ethyl-(2-et...,CCSCC[S+](CC)CCSP(=O)(OC)OC,CCSCC[S+](CC)CCSP(=O)(OC)OC,GICPMZCWCHOATC-UHFFFAOYSA-N,7.805466


In [9]:
df_ws.rename(columns={'Canonical SMILES': 'Canonical_SMILES'}, inplace=True)

##  Standardization  for work set

In [11]:
df_ws["Molecule"] = df_ws.apply(lambda x: standart(x.Canonical_SMILES), axis=1)
print('Kept data: ', len(df_ws), 'molecules')

Kept data:  1858 molecules


In [12]:
df_ws

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical_SMILES,InChIKey,rat_intravenous_LD50,Molecule
0,TOX-3990,178,acetamide,CC(N)=O,CC(N)=O,DLFVBJFMPXGRIB-UHFFFAOYSA-N,0.674442,<rdkit.Chem.rdchem.Mol object at 0x0000020A770...
1,TOX-2407,180,propan-2-one,CC(C)=O,CC(C)=O,CSCPPACGZOOCGX-UHFFFAOYSA-N,1.023664,<rdkit.Chem.rdchem.Mol object at 0x0000020A770...
2,TOX-8193,12025,pyrrolidin-2-one,O=C1CCCN1,O=C1CCCN1,HNJBEVLQSNELDL-UHFFFAOYSA-N,1.072628,<rdkit.Chem.rdchem.Mol object at 0x0000020A770...
3,TOX-4318,1030,"propane-1,2-diol",CC(O)CO,CC(O)CO,DNIAPMSPPWPWGF-UHFFFAOYSA-N,1.073618,<rdkit.Chem.rdchem.Mol object at 0x0000020A770...
4,TOX-8477,99937,"2,3,3a,5,6,6a-hexahydrofuro[3,2-b]furan-3,6-diol",OC1COC2C(O)COC12,OC1COC2C(O)COC12,KLDXJTOLSGUMSJ-UHFFFAOYSA-N,1.123382,<rdkit.Chem.rdchem.Mol object at 0x0000020A770...
...,...,...,...,...,...,...,...,...
1853,TOX-43158,116224,2-diethoxyphosphorylsulfanylethyl-ethyl-methyl...,CCOP(=O)(OCC)SCC[S+](C)CC,CCOP(=O)(OCC)SCC[S+](C)CC,YDYNMVNUGKEYOS-UHFFFAOYSA-N,7.232647,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...
1854,TOX-2075,14955,"9,10,21,25-tetramethoxy-15,15,30,30-tetramethy...",COc1ccc2cc1Oc1cc3c(cc1OC)CC[N+](C)(C)C3Cc1ccc(...,COc1ccc2cc1Oc1cc3c(cc1OC)CC[N+](C)(C)C3Cc1ccc(...,JFXBEKISTKFVAB-UHFFFAOYSA-N,7.270733,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...
1855,TOX-43154,116216,2-diethoxyphosphorylsulfanylethyl(diethyl)sulf...,CCOP(=O)(OCC)SCC[S+](CC)CC,CCOP(=O)(OCC)SCC[S+](CC)CC,QACHTXIQQRWDTM-UHFFFAOYSA-N,7.458497,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...
1856,TOX-43157,116222,2-dimethoxyphosphorylsulfanylethyl-ethyl-(2-et...,CCSCC[S+](CC)CCSP(=O)(OC)OC,CCSCC[S+](CC)CCSP(=O)(OC)OC,GICPMZCWCHOATC-UHFFFAOYSA-N,7.805466,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...


In [13]:
y_tr=df_ws.rat_intravenous_LD50		
y_tr

0       0.674442
1       1.023664
2       1.072628
3       1.073618
4       1.123382
          ...   
1853    7.232647
1854    7.270733
1855    7.458497
1856    7.805466
1857    7.842020
Name: rat_intravenous_LD50, Length: 1858, dtype: float64

In [14]:
moldf_ws=df_ws.Molecule

##  Load data and curation test set

In [16]:
df_ts=pd.read_csv('datasets/rat_LD50_TS.csv')
df_ts

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical_SMILES,InChIKey,rat_intravenous_LD50
0,TOX-69105,9887295,"1-[6-(benzenesulfonyl)-3-hydroxy-2,2-dimethyl-...",CC1(C)Oc2ccc(S(=O)(=O)c3ccccc3)cc2C(N2CCCC2=O)C1O,CC1(C)Oc2ccc(S(=O)(=O)c3ccccc3)cc2C(N2CCCC2=O)C1O,LKAQWOWWTKFLNX-UHFFFAOYSA-N,0.496458
1,TOX-5811,8172,2-[2-(2-hydroxyethoxy)ethoxy]ethanol,OCCOCCOCCO,OCCOCCOCCO,ZIBGPFATKBEMQZ-UHFFFAOYSA-N,1.108409
2,TOX-2835,4101,"1,3,5,7-tetrazatricyclo[3.3.1.13,7]decane",C1N2CN3CN1CN(C2)C3,C1N2CN3CN1CN(C2)C3,VKYKSIONXSXAKP-UHFFFAOYSA-N,1.182929
3,TOX-3906,174,"ethane-1,2-diol",OCCO,OCCO,LYCAIKOWRPUZTN-UHFFFAOYSA-N,1.279650
4,TOX-5749,8087,1-(2-hydroxypropoxy)propan-2-ol,CC(O)COCC(C)O,CC(O)COCC(C)O,AZUXKVXMJOIAOF-UHFFFAOYSA-N,1.364244
...,...,...,...,...,...,...,...
460,TOX-44743,76419053,"[4-[4-[4-[3,5-dihydroxy-6-(hydroxymethyl)-4-[3...",CC1(C)CCC2(C(=O)OC3OC(CO)C(O)C(OC4OC(CO)C(O)C(...,CC1(C)CCC2(C(=O)OC3OC(CO)C(O)C(OC4OC(CO)C(O)C(...,UZQJVUCHXGYFLQ-UHFFFAOYSA-N,6.425276
461,TOX-5571,7871,2-[fluoro(methyl)phosphoryl]oxypropane,CC(C)OP(C)(=O)F,CC(C)OP(C)(=O)F,DYAHQFWOVKZOOW-UHFFFAOYSA-N,6.555355
462,TOX-5132,7305,"3-[fluoro(methyl)phosphoryl]oxy-2,2-dimethylbu...",CC(OP(C)(=O)F)C(C)(C)C,CC(OP(C)(=O)F)C(C)(C)C,GRXKLBBBQUKJJZ-UHFFFAOYSA-N,6.612129
463,TOX-23193,102302,"2-[ethoxy(methyl)phosphoryl]sulfanyl-N,N-dimet...",CCOP(C)(=O)SCCN(C)C,CCOP(C)(=O)SCCN(C)C,PKDYQTANBZBIRM-UHFFFAOYSA-N,7.094383


##  Standardization  for test set

In [18]:
df_ts["Molecule"] = df_ts.apply(lambda x: standart(x.Canonical_SMILES), axis=1)
moldf_ts=df_ts[df_ts['SMILES']!='wrong_smiles']
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  465 molecules


In [19]:
moldf_ts

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical_SMILES,InChIKey,rat_intravenous_LD50,Molecule
0,TOX-69105,9887295,"1-[6-(benzenesulfonyl)-3-hydroxy-2,2-dimethyl-...",CC1(C)Oc2ccc(S(=O)(=O)c3ccccc3)cc2C(N2CCCC2=O)C1O,CC1(C)Oc2ccc(S(=O)(=O)c3ccccc3)cc2C(N2CCCC2=O)C1O,LKAQWOWWTKFLNX-UHFFFAOYSA-N,0.496458,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...
1,TOX-5811,8172,2-[2-(2-hydroxyethoxy)ethoxy]ethanol,OCCOCCOCCO,OCCOCCOCCO,ZIBGPFATKBEMQZ-UHFFFAOYSA-N,1.108409,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...
2,TOX-2835,4101,"1,3,5,7-tetrazatricyclo[3.3.1.13,7]decane",C1N2CN3CN1CN(C2)C3,C1N2CN3CN1CN(C2)C3,VKYKSIONXSXAKP-UHFFFAOYSA-N,1.182929,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...
3,TOX-3906,174,"ethane-1,2-diol",OCCO,OCCO,LYCAIKOWRPUZTN-UHFFFAOYSA-N,1.279650,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...
4,TOX-5749,8087,1-(2-hydroxypropoxy)propan-2-ol,CC(O)COCC(C)O,CC(O)COCC(C)O,AZUXKVXMJOIAOF-UHFFFAOYSA-N,1.364244,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...
...,...,...,...,...,...,...,...,...
460,TOX-44743,76419053,"[4-[4-[4-[3,5-dihydroxy-6-(hydroxymethyl)-4-[3...",CC1(C)CCC2(C(=O)OC3OC(CO)C(O)C(OC4OC(CO)C(O)C(...,CC1(C)CCC2(C(=O)OC3OC(CO)C(O)C(OC4OC(CO)C(O)C(...,UZQJVUCHXGYFLQ-UHFFFAOYSA-N,6.425276,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...
461,TOX-5571,7871,2-[fluoro(methyl)phosphoryl]oxypropane,CC(C)OP(C)(=O)F,CC(C)OP(C)(=O)F,DYAHQFWOVKZOOW-UHFFFAOYSA-N,6.555355,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...
462,TOX-5132,7305,"3-[fluoro(methyl)phosphoryl]oxy-2,2-dimethylbu...",CC(OP(C)(=O)F)C(C)(C)C,CC(OP(C)(=O)F)C(C)(C)C,GRXKLBBBQUKJJZ-UHFFFAOYSA-N,6.612129,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...
463,TOX-23193,102302,"2-[ethoxy(methyl)phosphoryl]sulfanyl-N,N-dimet...",CCOP(C)(=O)SCCN(C)C,CCOP(C)(=O)SCCN(C)C,PKDYQTANBZBIRM-UHFFFAOYSA-N,7.094383,<rdkit.Chem.rdchem.Mol object at 0x0000020A771...


In [20]:
y_ts=moldf_ts.rat_intravenous_LD50
y_ts

0      0.496458
1      1.108409
2      1.182929
3      1.279650
4      1.364244
         ...   
460    6.425276
461    6.555355
462    6.612129
463    7.094383
464    7.582023
Name: rat_intravenous_LD50, Length: 465, dtype: float64

In [21]:
moldf_ts=moldf_ts.Molecule

## Calculation MorganFingerprint for work set

In [23]:
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=1024)

In [24]:
fp_tr = [mfpgen.GetFingerprint(m)for m in moldf_ws]

In [25]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [26]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [27]:
savetxt('Models/MorganFingerprint/x_tr.csv', x_tr, delimiter=',')

In [28]:
x_tr.shape

(1858, 1024)

## Calculation MorganFingerprint for test set

In [30]:
fp_ts = [mfpgen.GetFingerprint(m) for m in moldf_ts]

In [31]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [32]:
x_ts = rdkit_numpy_convert(fp_ts)

In [33]:
x_ts.shape

(465, 1024)

In [72]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

# CatBoostRegressor

In [72]:
cv=KFold(n_splits=5, random_state=42, shuffle=True)

In [77]:
%%time
model = CatBoostRegressor()
parameters = {'depth' : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations'    : [100,500, 1000]
              }

grid = GridSearchCV(estimator=model, param_grid = parameters, n_jobs=-1, cv = cv)
grid.fit(x_tr, y_tr, verbose=False)

CPU times: total: 6min 36s
Wall time: 11min 38s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=<catboost.core.CatBoostRegressor object at 0x000001C3B96B3D70>,
             n_jobs=-1,
             param_grid={'depth': [6, 8, 10], 'iterations': [100, 500, 1000],
                         'learning_rate': [0.01, 0.05, 0.1]})

In [78]:
best_CatBR = grid.best_estimator_

In [79]:
grid.best_params_

{'depth': 10, 'iterations': 1000, 'learning_rate': 0.05}

In [80]:
y_pred_ws_GBR = best_CatBR.predict(x_tr)

In [81]:
R2_WS = round(r2_score(y_tr, y_pred_ws_GBR), 2)
R2_WS

0.97

In [82]:
RMSE_WS=round(np.sqrt(mean_squared_error(y_tr, y_pred_ws_GBR)), 2)
RMSE_WS

0.18

In [70]:
params={'verbose': False}

In [74]:
%%time
y_pred_CV_CatBR = cross_val_predict(best_CatBR, x_tr, y_tr, cv=cv, params=params)

CPU times: total: 21min 24s
Wall time: 2min 40s


In [84]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_CatBR), 2)
Q2_CV

0.56

In [85]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_CatBR)), 2)
RMSE_CV

0.65

# save the model to disk

In [87]:
pickle.dump(best_CatBR, open('Models/MorganFingerprint/Toxicity_CatBoost_MF.pkl', 'wb'))

# load the model from disk

In [58]:
best_CatBR = pickle.load(open('Models/MorganFingerprint/Toxicity_CatBoost_MF.pkl', 'rb'))

# 9. Prediction for test set's molecules

In [61]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [63]:
y_pred_GBR = best_CatBR.predict(x_ts)

In [65]:
Q2_TS = round(r2_score(y_ts, y_pred_GBR), 2)
Q2_TS

0.53

In [67]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_GBR)), 2)
RMSE_TS

0.68

# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [94]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [95]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.000000,2.000000,3.872983,2.449490,3.464102,3.000000,4.472136,2.236068,4.795832,2.449490,...,3.464102,3.741657,4.582576,4.898980,8.246211,3.000000,3.464102,2.645751,3.000000,2.645751
2,2.236068,2.236068,4.000000,3.000000,3.605551,3.316625,4.690416,2.236068,4.898980,2.449490,...,4.898980,8.185352,6.164414,5.291502,8.246211,3.000000,5.291502,3.000000,3.316625,3.000000
3,2.645751,2.236068,4.123106,3.000000,3.741657,3.464102,4.795832,2.449490,5.000000,2.645751,...,5.656854,8.185352,9.591663,6.164414,8.306623,3.741657,6.480741,3.605551,3.741657,3.316625
4,2.645751,2.449490,4.242640,3.162278,4.000000,3.605551,4.898980,2.449490,5.000000,3.162278,...,6.324555,8.185352,9.848858,7.416198,8.306623,3.741657,7.141428,3.741657,4.000000,3.741657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,10.677078,10.770329,10.908712,10.583005,10.816654,10.908712,10.954452,10.816654,10.770329,11.045361,...,12.206555,12.288206,13.000000,12.489996,12.041595,11.661903,12.165525,11.532562,11.532562,11.575837
1854,11.180340,11.269427,11.489125,11.180340,11.401754,11.401754,11.224972,11.313708,11.269427,11.445523,...,12.247449,12.409674,13.416408,12.609520,12.041595,12.041595,12.288206,12.000000,12.083046,12.124355
1855,11.357817,11.445523,11.832160,11.357817,11.618950,11.445523,11.489125,11.575837,11.532562,11.661903,...,12.569805,12.884099,13.416408,12.961481,12.449900,12.124355,12.806249,12.000000,12.165525,12.206555
1856,11.489125,11.575837,11.958261,11.489125,11.661903,11.489125,11.661903,11.618950,11.747340,11.704700,...,12.845233,13.038404,13.564660,13.114877,12.727922,12.165525,12.922848,12.124355,12.206555,12.247449


In [96]:
similarity= neighbors_k

In [97]:
Dmean=np.mean(similarity[1,:])

In [98]:
round(Dmean, 2)

4.24

In [99]:
std=np.std(similarity[1,:])

In [100]:
round(std, 2)

1.4

In [101]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.94


In [102]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [103]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,455,456,457,458,459,460,461,462,463,464
0,6.557438,1.000000,2.000000,2.236068,2.645751,5.916080,1.732051,4.690416,3.316625,2.828427,...,3.872983,5.291502,2.449490,5.916080,7.141428,6.403124,3.872983,4.358899,4.358899,4.690416
1,6.633250,2.236068,3.000000,2.449490,3.000000,6.000000,4.000000,5.099020,3.316625,3.162278,...,5.000000,5.567764,3.605551,5.916080,7.211102,6.928203,3.872983,4.358899,4.358899,4.690416
2,6.855655,2.645751,3.000000,2.645751,3.000000,6.000000,4.123106,5.196152,3.464102,3.605551,...,5.099020,5.830952,3.872983,6.000000,7.280110,7.745967,4.123106,4.582576,4.472136,4.795832
3,6.928203,2.828427,3.162278,2.645751,3.464102,6.244998,4.242640,5.291502,4.123106,4.000000,...,7.141428,5.916080,4.000000,6.000000,7.280110,7.937254,4.242640,4.582576,4.582576,5.000000
4,6.928203,3.316625,3.162278,2.828427,3.741657,6.244998,4.358899,5.291502,4.358899,4.472136,...,7.141428,6.244998,4.123106,6.082763,7.348469,8.062258,4.242640,4.690416,4.690416,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,11.401754,11.090536,10.954452,10.816654,10.816654,11.269427,10.862781,11.180340,11.224972,11.489125,...,11.704700,11.269427,11.661903,11.045361,11.958261,12.206555,11.135529,11.313708,11.532562,11.401754
1854,11.789826,11.489125,11.532562,11.313708,11.224972,11.401754,11.532562,11.401754,11.618950,11.958261,...,12.247449,11.661903,12.083046,11.958261,12.000000,12.288206,11.618950,11.789826,12.000000,11.789826
1855,12.449900,11.704700,11.704700,11.445523,11.401754,12.288206,11.874342,11.916375,11.789826,12.000000,...,12.409674,12.247449,12.124355,11.958261,12.529964,12.288206,11.789826,11.789826,12.083046,11.916375
1856,12.609520,11.747340,11.747340,11.489125,11.532562,12.489996,12.000000,12.083046,11.832160,12.041595,...,12.529964,12.247449,12.124355,12.247449,12.767145,12.328828,11.832160,12.000000,12.124355,11.958261


In [104]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[6.557 1.    2.    2.236 2.646 5.916 1.732 4.69  3.317 2.828 3.606 4.583
 3.    4.796 4.472 3.464 5.385 0.    4.243 3.162 5.477 5.831 6.    4.472
 3.873 4.243 3.162 4.899 2.236 4.    3.162 3.    5.745 4.583 7.681 3.873
 6.633 5.916 5.292 4.472 5.099 3.317 4.359 3.    7.81  1.    4.583 3.317
 3.317 2.236 3.162 3.162 3.464 5.099 3.    2.646 3.873 5.568 5.196 3.
 5.099 3.    6.    5.745 3.464 6.    5.099 3.317 6.    6.083 3.317 6.782
 3.606 4.    5.385 5.099 4.    6.325 1.732 4.583 4.    3.873 0.    2.449
 3.873 5.477 4.583 5.292 4.243 3.    3.162 4.243 5.745 3.464 3.464 4.472
 3.606 3.317 2.646 4.583 2.828 3.606 6.245 5.477 4.69  3.873 5.099 4.796
 4.899 4.    4.472 4.243 4.472 7.141 4.472 4.123 5.196 2.828 4.243 5.
 5.745 3.    3.873 4.243 3.742 3.742 3.464 3.    4.796 5.099 3.    2.828
 4.583 4.69  2.828 5.292 3.606 6.403 5.568 3.317 4.359 3.742 5.568 2.828
 5.477 4.796 5.385 5.292 4.359 3.742 2.828 4.796 4.    4.123 3.873 4.123
 4.    5.657 4.69  3.464 5.477 4.472 4.69  3.873 6.481 5.

In [105]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[False  True  True  True  True False  True  True  True  True  True  True
  True  True  True  True False  True  True  True False False False  True
  True  True  True  True  True  True  True  True False  True False  True
 False False False  True False  True  True  True False  True  True  True
  True  True  True  True  True False  True  True  True False False  True
 False  True False False  True False False  True False False  True False
  True  True False False  True False  True  True  True  True  True  True
  True False  True False  True  True  True  True False  True  True  True
  True  True  True  True  True  True False False  True  True False  True
  True  True  True  True  True False  True  True False  True  True False
 False  True  True  True  True  True  True  True  True False  True  True
  True  True  True False  True False False  True  True  True False  True
 False  True False False  True  True  True  True  True  True  True  True
  True False  True  True False  True  True  True Fa

In [106]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.72


In [107]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  1   2   3   4   6   7   8   9  10  11  12  13  14  15  17  18  19  23
  24  25  26  27  28  29  30  31  33  35  39  41  42  43  45  46  47  48
  49  50  51  52  54  55  56  59  61  64  67  70  72  73  76  78  79  80
  81  82  83  84  86  88  89  90  91  93  94  95  96  97  98  99 100 101
 104 105 107 108 109 110 111 112 114 115 117 118 121 122 123 124 125 126
 127 128 130 131 132 133 134 136 139 140 141 143 145 148 149 150 151 152
 153 154 155 156 158 159 161 162 163 166 167 168 169 171 172 173 174 175
 177 178 180 181 182 186 188 189 190 191 192 193 194 195 196 197 198 200
 201 202 203 204 205 206 207 209 210 211 213 215 216 217 219 220 223 224
 225 226 227 229 230 231 233 234 236 237 238 239 241 242 243 244 246 247
 248 249 250 251 253 254 255 256 258 259 260 263 264 266 268 269 271 272
 273 274 275 278 280 281 282 283 284 285 286 287 288 291 292 293 294 295
 296 297 300 303 304 305 307 308 309 311 312 313 314 315 316 317 318 320
 321 322 32

In [108]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [104]:
y_pred_GBR_ad=list(y_pred_GBR)

In [105]:
y_pred_GBR_ad[:] = [x for i,x in enumerate(y_pred_GBR_ad) if i not in out_Ad]

In [106]:
len(y_pred_GBR_ad)

1400

In [107]:
y_ts_ad=list(y_ts)

In [108]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [109]:
len(y_ts_ad)

1400

In [110]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_GBR_ad), 2)
Q2_TS

0.64

In [111]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_GBR_ad)), 2)
RMSE_TS

0.56

# SVM model building and validation

In [139]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [141]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [143]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [145]:
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(epsilon=0.2), n_jobs=-1,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [146]:
svm.best_params_
best_svm = svm.best_estimator_

In [147]:
svm.best_params_

{'C': 10, 'gamma': 0.01}

In [148]:
y_pred_ws_svm = best_svm.predict(x_tr)

In [149]:
R2_WS = round(r2_score(y_tr, y_pred_ws_svm), 2)
R2_WS

0.94

In [150]:
RMSE_WS=round(np.sqrt(mean_squared_error(y_tr, y_pred_ws_svm)), 2)
RMSE_WS

0.25

In [151]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [152]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 2)
Q2_CV

0.51

In [153]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.69

# 9. Prediction for test set's molecules

In [155]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [156]:
y_pred_svm = best_svm.predict(x_ts)

In [157]:
Q2_TS = round(r2_score(y_ts, y_pred_svm), 2)
Q2_TS

0.48

In [158]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_svm)), 2)
RMSE_TS

0.71

save the model to disk

In [255]:
pickle.dump(best_svm, open('models/MorganFingerprint/Toxicity_SVM_MF.pkl', 'wb'))

load the model from disk

In [53]:
best_svm = pickle.load(open('Models/Morgan_fingerprint/Toxicity_SVM_MF.pkl', 'rb'))

# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [208]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [209]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.000000,2.000000,3.872983,2.449490,3.464102,3.000000,4.472136,2.236068,4.795832,2.449490,...,3.464102,3.741657,4.582576,4.898980,8.246211,3.000000,3.464102,2.645751,3.000000,2.645751
2,2.236068,2.236068,4.000000,3.000000,3.605551,3.316625,4.690416,2.236068,4.898980,2.449490,...,4.898980,8.185352,6.164414,5.291502,8.246211,3.000000,5.291502,3.000000,3.316625,3.000000
3,2.645751,2.236068,4.123106,3.000000,3.741657,3.464102,4.795832,2.449490,5.000000,2.645751,...,5.656854,8.185352,9.591663,6.164414,8.306623,3.741657,6.480741,3.605551,3.741657,3.316625
4,2.645751,2.449490,4.242640,3.162278,4.000000,3.605551,4.898980,2.449490,5.000000,3.162278,...,6.324555,8.185352,9.848858,7.416198,8.306623,3.741657,7.141428,3.741657,4.000000,3.741657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,10.677078,10.770329,10.908712,10.583005,10.816654,10.908712,10.954452,10.816654,10.770329,11.045361,...,12.206555,12.288206,13.000000,12.489996,12.041595,11.661903,12.165525,11.532562,11.532562,11.575837
1854,11.180340,11.269427,11.489125,11.180340,11.401754,11.401754,11.224972,11.313708,11.269427,11.445523,...,12.247449,12.409674,13.416408,12.609520,12.041595,12.041595,12.288206,12.000000,12.083046,12.124355
1855,11.357817,11.445523,11.832160,11.357817,11.618950,11.445523,11.489125,11.575837,11.532562,11.661903,...,12.569805,12.884099,13.416408,12.961481,12.449900,12.124355,12.806249,12.000000,12.165525,12.206555
1856,11.489125,11.575837,11.958261,11.489125,11.661903,11.489125,11.661903,11.618950,11.747340,11.704700,...,12.845233,13.038404,13.564660,13.114877,12.727922,12.165525,12.922848,12.124355,12.206555,12.247449


In [212]:
similarity= neighbors_k

In [214]:
Dmean=np.mean(similarity[1,:])

In [216]:
round(Dmean, 2)

4.24

In [218]:
std=np.std(similarity[1,:])

In [220]:
round(std, 2)

1.4

In [222]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.94


In [224]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [226]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,455,456,457,458,459,460,461,462,463,464
0,6.557438,1.000000,2.000000,2.236068,2.645751,5.916080,1.732051,4.690416,3.316625,2.828427,...,3.872983,5.291502,2.449490,5.916080,7.141428,6.403124,3.872983,4.358899,4.358899,4.690416
1,6.633250,2.236068,3.000000,2.449490,3.000000,6.000000,4.000000,5.099020,3.316625,3.162278,...,5.000000,5.567764,3.605551,5.916080,7.211102,6.928203,3.872983,4.358899,4.358899,4.690416
2,6.855655,2.645751,3.000000,2.645751,3.000000,6.000000,4.123106,5.196152,3.464102,3.605551,...,5.099020,5.830952,3.872983,6.000000,7.280110,7.745967,4.123106,4.582576,4.472136,4.795832
3,6.928203,2.828427,3.162278,2.645751,3.464102,6.244998,4.242640,5.291502,4.123106,4.000000,...,7.141428,5.916080,4.000000,6.000000,7.280110,7.937254,4.242640,4.582576,4.582576,5.000000
4,6.928203,3.316625,3.162278,2.828427,3.741657,6.244998,4.358899,5.291502,4.358899,4.472136,...,7.141428,6.244998,4.123106,6.082763,7.348469,8.062258,4.242640,4.690416,4.690416,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,11.401754,11.090536,10.954452,10.816654,10.816654,11.269427,10.862781,11.180340,11.224972,11.489125,...,11.704700,11.269427,11.661903,11.045361,11.958261,12.206555,11.135529,11.313708,11.532562,11.401754
1854,11.789826,11.489125,11.532562,11.313708,11.224972,11.401754,11.532562,11.401754,11.618950,11.958261,...,12.247449,11.661903,12.083046,11.958261,12.000000,12.288206,11.618950,11.789826,12.000000,11.789826
1855,12.449900,11.704700,11.704700,11.445523,11.401754,12.288206,11.874342,11.916375,11.789826,12.000000,...,12.409674,12.247449,12.124355,11.958261,12.529964,12.288206,11.789826,11.789826,12.083046,11.916375
1856,12.609520,11.747340,11.747340,11.489125,11.532562,12.489996,12.000000,12.083046,11.832160,12.041595,...,12.529964,12.247449,12.124355,12.247449,12.767145,12.328828,11.832160,12.000000,12.124355,11.958261


In [228]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[6.557 1.    2.    2.236 2.646 5.916 1.732 4.69  3.317 2.828 3.606 4.583
 3.    4.796 4.472 3.464 5.385 0.    4.243 3.162 5.477 5.831 6.    4.472
 3.873 4.243 3.162 4.899 2.236 4.    3.162 3.    5.745 4.583 7.681 3.873
 6.633 5.916 5.292 4.472 5.099 3.317 4.359 3.    7.81  1.    4.583 3.317
 3.317 2.236 3.162 3.162 3.464 5.099 3.    2.646 3.873 5.568 5.196 3.
 5.099 3.    6.    5.745 3.464 6.    5.099 3.317 6.    6.083 3.317 6.782
 3.606 4.    5.385 5.099 4.    6.325 1.732 4.583 4.    3.873 0.    2.449
 3.873 5.477 4.583 5.292 4.243 3.    3.162 4.243 5.745 3.464 3.464 4.472
 3.606 3.317 2.646 4.583 2.828 3.606 6.245 5.477 4.69  3.873 5.099 4.796
 4.899 4.    4.472 4.243 4.472 7.141 4.472 4.123 5.196 2.828 4.243 5.
 5.745 3.    3.873 4.243 3.742 3.742 3.464 3.    4.796 5.099 3.    2.828
 4.583 4.69  2.828 5.292 3.606 6.403 5.568 3.317 4.359 3.742 5.568 2.828
 5.477 4.796 5.385 5.292 4.359 3.742 2.828 4.796 4.    4.123 3.873 4.123
 4.    5.657 4.69  3.464 5.477 4.472 4.69  3.873 6.481 5.

In [230]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[False  True  True  True  True False  True  True  True  True  True  True
  True  True  True  True False  True  True  True False False False  True
  True  True  True  True  True  True  True  True False  True False  True
 False False False  True False  True  True  True False  True  True  True
  True  True  True  True  True False  True  True  True False False  True
 False  True False False  True False False  True False False  True False
  True  True False False  True False  True  True  True  True  True  True
  True False  True False  True  True  True  True False  True  True  True
  True  True  True  True  True  True False False  True  True False  True
  True  True  True  True  True False  True  True False  True  True False
 False  True  True  True  True  True  True  True  True False  True  True
  True  True  True False  True False False  True  True  True False  True
 False  True False False  True  True  True  True  True  True  True  True
  True False  True  True False  True  True  True Fa

In [232]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.72


In [234]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  1   2   3   4   6   7   8   9  10  11  12  13  14  15  17  18  19  23
  24  25  26  27  28  29  30  31  33  35  39  41  42  43  45  46  47  48
  49  50  51  52  54  55  56  59  61  64  67  70  72  73  76  78  79  80
  81  82  83  84  86  88  89  90  91  93  94  95  96  97  98  99 100 101
 104 105 107 108 109 110 111 112 114 115 117 118 121 122 123 124 125 126
 127 128 130 131 132 133 134 136 139 140 141 143 145 148 149 150 151 152
 153 154 155 156 158 159 161 162 163 166 167 168 169 171 172 173 174 175
 177 178 180 181 182 186 188 189 190 191 192 193 194 195 196 197 198 200
 201 202 203 204 205 206 207 209 210 211 213 215 216 217 219 220 223 224
 225 226 227 229 230 231 233 234 236 237 238 239 241 242 243 244 246 247
 248 249 250 251 253 254 255 256 258 259 260 263 264 266 268 269 271 272
 273 274 275 278 280 281 282 283 284 285 286 287 288 291 292 293 294 295
 296 297 300 303 304 305 307 308 309 311 312 313 314 315 316 317 318 320
 321 322 32

In [236]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [239]:
y_pred_svm_ad=list(y_pred_svm)

In [241]:
y_pred_svm_ad[:] = [x for i,x in enumerate(y_pred_svm_ad) if i not in out_Ad]

In [243]:
len(y_pred_svm_ad)

336

In [245]:
y_ts_ad=list(y_ts)

In [247]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [249]:
len(y_ts_ad)

336

In [251]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_svm_ad), 2)
Q2_TS

0.56

In [253]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_svm_ad)), 2)
RMSE_TS

0.64

# Multi-layer Perceptron regressor

In [258]:
from sklearn.neural_network import MLPRegressor

In [260]:
param_grid ={"hidden_layer_sizes": [(400, 300, 200, 100),(100, 100, 100), (10, 10, 10),(50,)], "activation": ["tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005], 'max_iter': [1000, 2000]}

In [262]:
m = GridSearchCV(MLPRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [264]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=MLPRegressor(), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [5e-05, 0.0005],
                         'hidden_layer_sizes': [(400, 300, 200, 100),
                                                (100, 100, 100), (10, 10, 10),
                                                (50,)],
                         'max_iter': [1000, 2000],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=1)

In [265]:
best_MLPR = m.best_estimator_

In [266]:
m.best_params_

{'activation': 'relu',
 'alpha': 0.0005,
 'hidden_layer_sizes': (400, 300, 200, 100),
 'max_iter': 1000,
 'solver': 'adam'}

In [267]:
y_pred_ws_MLPR = best_MLPR.predict(x_tr)

In [268]:
R2_WS = round(r2_score(y_tr, y_pred_ws_MLPR), 2)
R2_WS

1.0

In [269]:
RMSE_WS=round(np.sqrt(mean_squared_error(y_tr, y_pred_ws_MLPR)), 2)
RMSE_WS

0.06

In [270]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, x_tr, y_tr, cv=cv)

In [271]:
y_pred_CV_MLPR

array([2.0441778, 1.680971 , 2.7681153, ..., 7.328272 , 7.2295475,
       8.2099   ], dtype=float32)

In [272]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_MLPR), 2)
Q2_CV

0.49

In [273]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_MLPR)), 2)
RMSE_CV

0.7

# 9. Prediction for test set's molecules

In [275]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [276]:
y_pred_MLPR = best_MLPR.predict(x_ts)

In [277]:
Q2_TS = round(r2_score(y_ts, y_pred_MLPR), 2)
Q2_TS

0.46

In [278]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_MLPR)), 2)
RMSE_TS

0.73

# save the model to disk

In [280]:
pickle.dump(best_MLPR, open('models/MorganFingerprint/Toxicity_MLPR_MF.pkl', 'wb'))

# load the model from disk

In [24]:
best_MLPR = pickle.load(open('models/MorganFingerprint/Toxicity_MLPR_MF.pkl', 'rb'))

# 11. Estimating applicability domain. Method - Euclidian distances, K=1

In [282]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [283]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.000000,2.000000,3.872983,2.449490,3.464102,3.000000,4.472136,2.236068,4.795832,2.449490,...,3.464102,3.741657,4.582576,4.898980,8.246211,3.000000,3.464102,2.645751,3.000000,2.645751
2,2.236068,2.236068,4.000000,3.000000,3.605551,3.316625,4.690416,2.236068,4.898980,2.449490,...,4.898980,8.185352,6.164414,5.291502,8.246211,3.000000,5.291502,3.000000,3.316625,3.000000
3,2.645751,2.236068,4.123106,3.000000,3.741657,3.464102,4.795832,2.449490,5.000000,2.645751,...,5.656854,8.185352,9.591663,6.164414,8.306623,3.741657,6.480741,3.605551,3.741657,3.316625
4,2.645751,2.449490,4.242640,3.162278,4.000000,3.605551,4.898980,2.449490,5.000000,3.162278,...,6.324555,8.185352,9.848858,7.416198,8.306623,3.741657,7.141428,3.741657,4.000000,3.741657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,10.677078,10.770329,10.908712,10.583005,10.816654,10.908712,10.954452,10.816654,10.770329,11.045361,...,12.206555,12.288206,13.000000,12.489996,12.041595,11.661903,12.165525,11.532562,11.532562,11.575837
1854,11.180340,11.269427,11.489125,11.180340,11.401754,11.401754,11.224972,11.313708,11.269427,11.445523,...,12.247449,12.409674,13.416408,12.609520,12.041595,12.041595,12.288206,12.000000,12.083046,12.124355
1855,11.357817,11.445523,11.832160,11.357817,11.618950,11.445523,11.489125,11.575837,11.532562,11.661903,...,12.569805,12.884099,13.416408,12.961481,12.449900,12.124355,12.806249,12.000000,12.165525,12.206555
1856,11.489125,11.575837,11.958261,11.489125,11.661903,11.489125,11.661903,11.618950,11.747340,11.704700,...,12.845233,13.038404,13.564660,13.114877,12.727922,12.165525,12.922848,12.124355,12.206555,12.247449


In [284]:
similarity= neighbors_k

In [285]:
Dmean=np.mean(similarity[1,:])

In [286]:
round(Dmean, 2)

4.24

In [287]:
std=np.std(similarity[1,:])

In [288]:
round(std, 2)

1.4

In [289]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.94


In [290]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [291]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,455,456,457,458,459,460,461,462,463,464
0,6.557438,1.000000,2.000000,2.236068,2.645751,5.916080,1.732051,4.690416,3.316625,2.828427,...,3.872983,5.291502,2.449490,5.916080,7.141428,6.403124,3.872983,4.358899,4.358899,4.690416
1,6.633250,2.236068,3.000000,2.449490,3.000000,6.000000,4.000000,5.099020,3.316625,3.162278,...,5.000000,5.567764,3.605551,5.916080,7.211102,6.928203,3.872983,4.358899,4.358899,4.690416
2,6.855655,2.645751,3.000000,2.645751,3.000000,6.000000,4.123106,5.196152,3.464102,3.605551,...,5.099020,5.830952,3.872983,6.000000,7.280110,7.745967,4.123106,4.582576,4.472136,4.795832
3,6.928203,2.828427,3.162278,2.645751,3.464102,6.244998,4.242640,5.291502,4.123106,4.000000,...,7.141428,5.916080,4.000000,6.000000,7.280110,7.937254,4.242640,4.582576,4.582576,5.000000
4,6.928203,3.316625,3.162278,2.828427,3.741657,6.244998,4.358899,5.291502,4.358899,4.472136,...,7.141428,6.244998,4.123106,6.082763,7.348469,8.062258,4.242640,4.690416,4.690416,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,11.401754,11.090536,10.954452,10.816654,10.816654,11.269427,10.862781,11.180340,11.224972,11.489125,...,11.704700,11.269427,11.661903,11.045361,11.958261,12.206555,11.135529,11.313708,11.532562,11.401754
1854,11.789826,11.489125,11.532562,11.313708,11.224972,11.401754,11.532562,11.401754,11.618950,11.958261,...,12.247449,11.661903,12.083046,11.958261,12.000000,12.288206,11.618950,11.789826,12.000000,11.789826
1855,12.449900,11.704700,11.704700,11.445523,11.401754,12.288206,11.874342,11.916375,11.789826,12.000000,...,12.409674,12.247449,12.124355,11.958261,12.529964,12.288206,11.789826,11.789826,12.083046,11.916375
1856,12.609520,11.747340,11.747340,11.489125,11.532562,12.489996,12.000000,12.083046,11.832160,12.041595,...,12.529964,12.247449,12.124355,12.247449,12.767145,12.328828,11.832160,12.000000,12.124355,11.958261


In [292]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[6.557 1.    2.    2.236 2.646 5.916 1.732 4.69  3.317 2.828 3.606 4.583
 3.    4.796 4.472 3.464 5.385 0.    4.243 3.162 5.477 5.831 6.    4.472
 3.873 4.243 3.162 4.899 2.236 4.    3.162 3.    5.745 4.583 7.681 3.873
 6.633 5.916 5.292 4.472 5.099 3.317 4.359 3.    7.81  1.    4.583 3.317
 3.317 2.236 3.162 3.162 3.464 5.099 3.    2.646 3.873 5.568 5.196 3.
 5.099 3.    6.    5.745 3.464 6.    5.099 3.317 6.    6.083 3.317 6.782
 3.606 4.    5.385 5.099 4.    6.325 1.732 4.583 4.    3.873 0.    2.449
 3.873 5.477 4.583 5.292 4.243 3.    3.162 4.243 5.745 3.464 3.464 4.472
 3.606 3.317 2.646 4.583 2.828 3.606 6.245 5.477 4.69  3.873 5.099 4.796
 4.899 4.    4.472 4.243 4.472 7.141 4.472 4.123 5.196 2.828 4.243 5.
 5.745 3.    3.873 4.243 3.742 3.742 3.464 3.    4.796 5.099 3.    2.828
 4.583 4.69  2.828 5.292 3.606 6.403 5.568 3.317 4.359 3.742 5.568 2.828
 5.477 4.796 5.385 5.292 4.359 3.742 2.828 4.796 4.    4.123 3.873 4.123
 4.    5.657 4.69  3.464 5.477 4.472 4.69  3.873 6.481 5.

In [293]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[False  True  True  True  True False  True  True  True  True  True  True
  True  True  True  True False  True  True  True False False False  True
  True  True  True  True  True  True  True  True False  True False  True
 False False False  True False  True  True  True False  True  True  True
  True  True  True  True  True False  True  True  True False False  True
 False  True False False  True False False  True False False  True False
  True  True False False  True False  True  True  True  True  True  True
  True False  True False  True  True  True  True False  True  True  True
  True  True  True  True  True  True False False  True  True False  True
  True  True  True  True  True False  True  True False  True  True False
 False  True  True  True  True  True  True  True  True False  True  True
  True  True  True False  True False False  True  True  True False  True
 False  True False False  True  True  True  True  True  True  True  True
  True False  True  True False  True  True  True Fa

In [294]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.72


In [295]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  1   2   3   4   6   7   8   9  10  11  12  13  14  15  17  18  19  23
  24  25  26  27  28  29  30  31  33  35  39  41  42  43  45  46  47  48
  49  50  51  52  54  55  56  59  61  64  67  70  72  73  76  78  79  80
  81  82  83  84  86  88  89  90  91  93  94  95  96  97  98  99 100 101
 104 105 107 108 109 110 111 112 114 115 117 118 121 122 123 124 125 126
 127 128 130 131 132 133 134 136 139 140 141 143 145 148 149 150 151 152
 153 154 155 156 158 159 161 162 163 166 167 168 169 171 172 173 174 175
 177 178 180 181 182 186 188 189 190 191 192 193 194 195 196 197 198 200
 201 202 203 204 205 206 207 209 210 211 213 215 216 217 219 220 223 224
 225 226 227 229 230 231 233 234 236 237 238 239 241 242 243 244 246 247
 248 249 250 251 253 254 255 256 258 259 260 263 264 266 268 269 271 272
 273 274 275 278 280 281 282 283 284 285 286 287 288 291 292 293 294 295
 296 297 300 303 304 305 307 308 309 311 312 313 314 315 316 317 318 320
 321 322 32

In [296]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [298]:
y_pred_MLPR_ad=list(y_pred_MLPR)

In [299]:
y_pred_MLPR_ad[:] = [x for i,x in enumerate(y_pred_MLPR_ad) if i not in out_Ad]

In [300]:
len(y_pred_MLPR_ad)

336

In [301]:
y_ts_ad=list(y_ts)

In [302]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [303]:
len(y_ts_ad)

336

In [304]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_MLPR_ad), 2)
Q2_TS

0.53

In [305]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_MLPR_ad)), 2)
RMSE_TS

0.67

In [306]:
from sklearn.neighbors import KNeighborsRegressor

# k-nearest neighbors

In [308]:
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)

In [309]:
m = GridSearchCV(KNeighborsRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [310]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30]},
             verbose=1)

In [311]:
best_kNN = m.best_estimator_

In [312]:
m.best_params_

{'n_neighbors': 4}

In [313]:
y_pred_ws_kNN = best_kNN.predict(x_tr)

In [314]:
R2_WS = round(r2_score(y_tr, y_pred_ws_kNN), 2)
R2_WS

0.5

In [315]:
RMSE_WS=round(np.sqrt(mean_squared_error(y_tr, y_pred_ws_kNN)), 2)
RMSE_WS

0.69

In [316]:
y_pred_CV_kNN = cross_val_predict(best_kNN, x_tr, y_tr, cv=cv)

In [317]:
y_pred_CV_kNN

array([1.5565531, 1.4770201, 2.3826187, ..., 6.021414 , 6.4935565,
       6.0214143], dtype=float32)

In [318]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_kNN), 2)
Q2_CV

0.02

In [319]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_kNN)), 2)
RMSE_CV

0.97

# Prediction for test set's molecules

In [321]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [322]:
y_pred_kNN = best_kNN.predict(x_ts)

In [323]:
Q2_TS = round(r2_score(y_ts, y_pred_kNN), 2)
Q2_TS

-0.03

In [324]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_kNN)), 2)
RMSE_TS

1.01

# save the model to disk

In [326]:
pickle.dump(best_kNN, open('models/MorganFingerprint/Toxicity_kNN_MF.pkl', 'wb'))

# load the model from disk

In [164]:
best_kNN = pickle.load(open('models/MorganFingerprint/Toxicity_kNN_MF.pkl', 'rb'))

# Estimating applicability domain. Method - Euclidian distances, K=1

In [328]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [329]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.000000,2.000000,3.872983,2.449490,3.464102,3.000000,4.472136,2.236068,4.795832,2.449490,...,3.464102,3.741657,4.582576,4.898980,8.246211,3.000000,3.464102,2.645751,3.000000,2.645751
2,2.236068,2.236068,4.000000,3.000000,3.605551,3.316625,4.690416,2.236068,4.898980,2.449490,...,4.898980,8.185352,6.164414,5.291502,8.246211,3.000000,5.291502,3.000000,3.316625,3.000000
3,2.645751,2.236068,4.123106,3.000000,3.741657,3.464102,4.795832,2.449490,5.000000,2.645751,...,5.656854,8.185352,9.591663,6.164414,8.306623,3.741657,6.480741,3.605551,3.741657,3.316625
4,2.645751,2.449490,4.242640,3.162278,4.000000,3.605551,4.898980,2.449490,5.000000,3.162278,...,6.324555,8.185352,9.848858,7.416198,8.306623,3.741657,7.141428,3.741657,4.000000,3.741657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,10.677078,10.770329,10.908712,10.583005,10.816654,10.908712,10.954452,10.816654,10.770329,11.045361,...,12.206555,12.288206,13.000000,12.489996,12.041595,11.661903,12.165525,11.532562,11.532562,11.575837
1854,11.180340,11.269427,11.489125,11.180340,11.401754,11.401754,11.224972,11.313708,11.269427,11.445523,...,12.247449,12.409674,13.416408,12.609520,12.041595,12.041595,12.288206,12.000000,12.083046,12.124355
1855,11.357817,11.445523,11.832160,11.357817,11.618950,11.445523,11.489125,11.575837,11.532562,11.661903,...,12.569805,12.884099,13.416408,12.961481,12.449900,12.124355,12.806249,12.000000,12.165525,12.206555
1856,11.489125,11.575837,11.958261,11.489125,11.661903,11.489125,11.661903,11.618950,11.747340,11.704700,...,12.845233,13.038404,13.564660,13.114877,12.727922,12.165525,12.922848,12.124355,12.206555,12.247449


In [330]:
similarity= neighbors_k

In [331]:
Dmean=np.mean(similarity[1,:])

In [332]:
round(Dmean, 2)

4.24

In [333]:
std=np.std(similarity[1,:])

In [334]:
round(std, 2)

1.4

In [335]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.94


In [336]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [337]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,455,456,457,458,459,460,461,462,463,464
0,6.557438,1.000000,2.000000,2.236068,2.645751,5.916080,1.732051,4.690416,3.316625,2.828427,...,3.872983,5.291502,2.449490,5.916080,7.141428,6.403124,3.872983,4.358899,4.358899,4.690416
1,6.633250,2.236068,3.000000,2.449490,3.000000,6.000000,4.000000,5.099020,3.316625,3.162278,...,5.000000,5.567764,3.605551,5.916080,7.211102,6.928203,3.872983,4.358899,4.358899,4.690416
2,6.855655,2.645751,3.000000,2.645751,3.000000,6.000000,4.123106,5.196152,3.464102,3.605551,...,5.099020,5.830952,3.872983,6.000000,7.280110,7.745967,4.123106,4.582576,4.472136,4.795832
3,6.928203,2.828427,3.162278,2.645751,3.464102,6.244998,4.242640,5.291502,4.123106,4.000000,...,7.141428,5.916080,4.000000,6.000000,7.280110,7.937254,4.242640,4.582576,4.582576,5.000000
4,6.928203,3.316625,3.162278,2.828427,3.741657,6.244998,4.358899,5.291502,4.358899,4.472136,...,7.141428,6.244998,4.123106,6.082763,7.348469,8.062258,4.242640,4.690416,4.690416,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,11.401754,11.090536,10.954452,10.816654,10.816654,11.269427,10.862781,11.180340,11.224972,11.489125,...,11.704700,11.269427,11.661903,11.045361,11.958261,12.206555,11.135529,11.313708,11.532562,11.401754
1854,11.789826,11.489125,11.532562,11.313708,11.224972,11.401754,11.532562,11.401754,11.618950,11.958261,...,12.247449,11.661903,12.083046,11.958261,12.000000,12.288206,11.618950,11.789826,12.000000,11.789826
1855,12.449900,11.704700,11.704700,11.445523,11.401754,12.288206,11.874342,11.916375,11.789826,12.000000,...,12.409674,12.247449,12.124355,11.958261,12.529964,12.288206,11.789826,11.789826,12.083046,11.916375
1856,12.609520,11.747340,11.747340,11.489125,11.532562,12.489996,12.000000,12.083046,11.832160,12.041595,...,12.529964,12.247449,12.124355,12.247449,12.767145,12.328828,11.832160,12.000000,12.124355,11.958261


In [338]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[6.557 1.    2.    2.236 2.646 5.916 1.732 4.69  3.317 2.828 3.606 4.583
 3.    4.796 4.472 3.464 5.385 0.    4.243 3.162 5.477 5.831 6.    4.472
 3.873 4.243 3.162 4.899 2.236 4.    3.162 3.    5.745 4.583 7.681 3.873
 6.633 5.916 5.292 4.472 5.099 3.317 4.359 3.    7.81  1.    4.583 3.317
 3.317 2.236 3.162 3.162 3.464 5.099 3.    2.646 3.873 5.568 5.196 3.
 5.099 3.    6.    5.745 3.464 6.    5.099 3.317 6.    6.083 3.317 6.782
 3.606 4.    5.385 5.099 4.    6.325 1.732 4.583 4.    3.873 0.    2.449
 3.873 5.477 4.583 5.292 4.243 3.    3.162 4.243 5.745 3.464 3.464 4.472
 3.606 3.317 2.646 4.583 2.828 3.606 6.245 5.477 4.69  3.873 5.099 4.796
 4.899 4.    4.472 4.243 4.472 7.141 4.472 4.123 5.196 2.828 4.243 5.
 5.745 3.    3.873 4.243 3.742 3.742 3.464 3.    4.796 5.099 3.    2.828
 4.583 4.69  2.828 5.292 3.606 6.403 5.568 3.317 4.359 3.742 5.568 2.828
 5.477 4.796 5.385 5.292 4.359 3.742 2.828 4.796 4.    4.123 3.873 4.123
 4.    5.657 4.69  3.464 5.477 4.472 4.69  3.873 6.481 5.

In [339]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[False  True  True  True  True False  True  True  True  True  True  True
  True  True  True  True False  True  True  True False False False  True
  True  True  True  True  True  True  True  True False  True False  True
 False False False  True False  True  True  True False  True  True  True
  True  True  True  True  True False  True  True  True False False  True
 False  True False False  True False False  True False False  True False
  True  True False False  True False  True  True  True  True  True  True
  True False  True False  True  True  True  True False  True  True  True
  True  True  True  True  True  True False False  True  True False  True
  True  True  True  True  True False  True  True False  True  True False
 False  True  True  True  True  True  True  True  True False  True  True
  True  True  True False  True False False  True  True  True False  True
 False  True False False  True  True  True  True  True  True  True  True
  True False  True  True False  True  True  True Fa

In [340]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.72


In [341]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  1   2   3   4   6   7   8   9  10  11  12  13  14  15  17  18  19  23
  24  25  26  27  28  29  30  31  33  35  39  41  42  43  45  46  47  48
  49  50  51  52  54  55  56  59  61  64  67  70  72  73  76  78  79  80
  81  82  83  84  86  88  89  90  91  93  94  95  96  97  98  99 100 101
 104 105 107 108 109 110 111 112 114 115 117 118 121 122 123 124 125 126
 127 128 130 131 132 133 134 136 139 140 141 143 145 148 149 150 151 152
 153 154 155 156 158 159 161 162 163 166 167 168 169 171 172 173 174 175
 177 178 180 181 182 186 188 189 190 191 192 193 194 195 196 197 198 200
 201 202 203 204 205 206 207 209 210 211 213 215 216 217 219 220 223 224
 225 226 227 229 230 231 233 234 236 237 238 239 241 242 243 244 246 247
 248 249 250 251 253 254 255 256 258 259 260 263 264 266 268 269 271 272
 273 274 275 278 280 281 282 283 284 285 286 287 288 291 292 293 294 295
 296 297 300 303 304 305 307 308 309 311 312 313 314 315 316 317 318 320
 321 322 32

In [342]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [344]:
y_pred_kNN_ad=list(y_pred_kNN)

In [345]:
y_pred_kNN_ad[:] = [x for i,x in enumerate(y_pred_kNN_ad) if i not in out_Ad]

In [346]:
len(y_pred_kNN_ad)

336

In [347]:
y_ts_ad=list(y_ts)

In [348]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [349]:
len(y_ts_ad)

336

In [350]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_kNN_ad), 2)
Q2_TS

0.24

In [351]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_kNN_ad)), 2)
RMSE_TS

0.84